In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

In [2]:


# 1. 构建一个简单的数据集：预测序列中数字的总和
class ToyDataset(Dataset):
    def __init__(self, n_samples=1000, seq_len=10):
        self.data = []
        self.labels = []
        for _ in range(n_samples):
            x = np.random.randint(0, 10, seq_len)
            self.data.append(x)
            self.labels.append(np.sum(x))
        self.data = torch.LongTensor(self.data)
        self.labels = torch.FloatTensor(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 2. Positional Encoding（正余弦）
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # [1, max_len, d_model]

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# 3. Transformer Encoder-only 模型
class TransformerEncoderSum(nn.Module):
    def __init__(self, vocab_size=10, d_model=64, num_layers=2, nhead=4, max_len=10):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.regressor = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)                       # [B, L, D]
        x = self.pos_enc(x)
        x = self.encoder(x)
        x = x.mean(dim=1)                           # mean pooling
        return self.regressor(x).squeeze(-1)        # [B]

In [3]:
train_data = ToyDataset(800)
test_data = ToyDataset(200)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

C:\Users\86460\AppData\Local\Temp\ipykernel_77072\736188540.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  self.data = torch.LongTensor(self.data)


In [13]:
len(train_loader.dataset)

800

In [8]:
train_loader.dataset[0]

(tensor([6, 3, 7, 4, 6, 9, 2, 6, 7, 4]), tensor(54.))

In [14]:
# 4. 初始化模型和训练设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerEncoderSum().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# 5. 训练过程
for epoch in range(10):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}] Train Loss: {total_loss / len(train_loader):.4f}")

    # 测试集评估
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            test_loss += loss.item()
    print(f"           Test Loss : {test_loss / len(test_loader):.4f}")


[Epoch 1] Train Loss: 1694.3564
           Test Loss : 1566.7546
[Epoch 2] Train Loss: 1528.8568
           Test Loss : 1417.5214
[Epoch 3] Train Loss: 1367.6046
           Test Loss : 1254.8203
[Epoch 4] Train Loss: 1198.0830
           Test Loss : 1086.3816
[Epoch 5] Train Loss: 1025.0130
           Test Loss : 918.3771
[Epoch 6] Train Loss: 856.4308
           Test Loss : 757.6829
[Epoch 7] Train Loss: 699.2513
           Test Loss : 609.9579
[Epoch 8] Train Loss: 557.8762
           Test Loss : 479.7621
[Epoch 9] Train Loss: 435.2838
           Test Loss : 370.7890
[Epoch 10] Train Loss: 334.5609
           Test Loss : 282.7758


你这个问题问得非常好，而且是很多刚入门 Transformer 的人常有的疑惑。确实 `PositionalEncoding` 看起来像是“魔法”，但是理解它之后其实**非常优雅而简单**。

---

## 🧠 为什么要有 PositionalEncoding？

Transformer 本身**没有循环结构（不像 RNN）也没有卷积（不像 CNN）**，所以它**天然不知道序列中哪个 token 在第几个位置**。

如果不给它加位置信息，那么：

```text
["hello", "world"] 和 ["world", "hello"]
→ 对模型来说是一样的！
```

所以必须想办法把“**我是在第 t 个位置**”的信息告诉模型。

---

## 🧭 怎么告诉模型位置信息？

这就有两种思路：
1. **Learnable Position Embedding**（可以学习的参数）🧠
2. **Fixed Sinusoidal Positional Encoding**（固定的正余弦函数）🧮 ✅ STRAFE 和经典 Transformer 用的就是这个！

---

## 📐 正余弦 PositionalEncoding 原理（Sinusoidal PE）

论文《Attention is All You Need》中给出如下定义：

对于每个位置 \(pos\) 和每个维度 \(i\)，定义：

\[
\text{PE}_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
\]
\[
\text{PE}_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
\]

---

### ✅ 解读一下这个式子：

| 符号         | 含义                               |
|--------------|------------------------------------|
| `pos`        | 当前位置（第几个 token）           |
| `i`          | 当前维度（如 dim=0, 1, 2...）      |
| `d_model`    | 总的嵌入维度                       |
| `10000^{2i/d}` | 控制周期长短，让不同维度频率不同 |

它的本质是：**不同维度的 sin 和 cos 周期不同 → 模型可以组合出唯一的位置模式。**

---

## 🔍 为什么正余弦很酷？

1. **不用训练（可复现、轻量）**
2. **任意长序列都能处理（理论上不限 max_len）**
3. **相对位置可导出**：即 \( \text{PE}_{pos+k} - \text{PE}_{pos} \) 是固定模式！

---

## 👇 图示感受一下：

| Position | sin 曲线     | cos 曲线     |
|----------|--------------|--------------|
| 0        | sin(0)=0     | cos(0)=1     |
| 1        | sin(x)       | cos(x)       |
| 2        | sin(2x)      | cos(2x)      |
| ...      | ...          | ...          |

不同维度频率不一样，交叉叠加起来就像“位置的指纹”。

---

## ✅ PyTorch 手写 PositionalEncoding：

你看到的代码其实就是上面公式的 vectorized 实现：

```python
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数位 sin
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数位 cos
        self.register_buffer('pe', pe.unsqueeze(0))   # shape: [1, max_len, d_model]

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]
```

---

## 🧠 重点总结：

| 问题                          | 答案                                                                 |
|-------------------------------|----------------------------------------------------------------------|
| 为什么要 PositionalEncoding？ | Transformer 没有顺序感，要告诉它“位置”。                             |
| 为什么是 sin 和 cos？         | 可以固定生成，有周期性、区分性、支持任意长度，不用训练。             |
| 为什么手写？                  | PyTorch 官方没封装，只能我们手动写（或用 huggingface 的）。         |

---

## 📌 如果你不想写，也可以直接用 HuggingFace 提供的：

```python
from transformers import BertModel
model = BertModel.from_pretrained("bert-base-uncased")
# 它内部就有 learnable position embedding
```

---

需要我给你写一个 demo，把 sin-embedding 图画出来看看它的结构吗？还是你想试试 learnable 的 positional embedding 对比一下？😎

非常棒的问题！Transformer 里的 `nhead` 参数（即多头注意力的 "头数"）是整个注意力机制的核心调控参数之一，它**不会改变模型输出的维度 `d_model`，但会改变注意力机制的表达能力和并行建模能力**。

下面我来详细解释它的意义、原理、影响和使用建议👇

---

## ✅ 一句话解释：

> `nhead` 决定了 Transformer 中 **“多头注意力机制”的头数**，即同时有多少个注意力子空间（subspace）在并行地处理信息。

---

## 🧠 什么是 Multi-Head Attention？

Transformer 中的注意力机制是这样的：

```text
Input → Linear变换 → Query / Key / Value → Scaled Dot Product Attention
```

如果我们只用一个头（single-head attention），那所有的信息都集中在一个空间里学习。

但如果我们用 **多头（multi-head）**，我们就可以：

> 把 `d_model` 拆分成多个子空间，在多个注意力头上 **并行学习不同的注意力模式**，然后合并在一起。

---

## 🔍 举个例子：

假设你的 `d_model = 64`，你设定了 `nhead = 4`，那会发生什么？

- 每个 attention 头负责 64 / 4 = 16 维的子空间；
- 模型会生成 4 套 `Query/Key/Value`；
- 每个头独立做 attention 计算；
- 然后把 4 个输出拼接起来，最后用一个线性层投影回 64 维。

```text
[64维] →
  ├─ Head 1 (16维 attention)
  ├─ Head 2 (16维 attention)
  ├─ Head 3 (16维 attention)
  └─ Head 4 (16维 attention)
→ 拼接成 64维 → Linear → 输出
```

---

## ✅ `nhead` 的作用是什么？

| 作用                             | 说明                                                                 |
|----------------------------------|----------------------------------------------------------------------|
| **提升建模能力**                  | 每个头可以学习不同的关注模式（比如一个头关注前文，一个关注当前）         |
| **增强表达力**                    | 多头注意力等于多个低维空间的并行建模，能捕捉更多层次和粒度的依赖关系      |
| **支持并行计算**                  | 每个头可以独立计算 attention，可以在 GPU 上并行加速                     |
| **保持输出维度不变**              | 注意：`nhead` 不会改变最后的输出维度 `d_model`，只是内部划分多头计算       |

---

## 🧪 如果 nhead 设太小或太大会怎样？

| 设置            | 影响                                                |
|-----------------|-----------------------------------------------------|
| `nhead=1`        | 单一注意力模式，表达能力弱，容易 underfit            |
| `nhead=2~8`      | 通常比较平衡，能捕捉多种关系，主流模型采用这个范围  |
| `nhead 很大`     | 每头维度太小（d_model / nhead 变小），可能学不到足够信息；计算复杂度也上升

---

## 🧠 使用建议

- `d_model` 必须能被 `nhead` 整除（否则无法分头）
- 如果你用 `d_model = 64`，推荐：
  - `nhead=4` → 每头 16 维
  - `nhead=8` → 每头 8 维
- GPT-2 / BERT 等模型：
  - BERT-base：`d_model=768`, `nhead=12`
  - GPT-2-small：`d_model=768`, `nhead=12`

---

## 📌 总结

| 参数      | 含义                                  |
|-----------|---------------------------------------|
| `nhead`   | Multi-head Attention 的头数            |
| 作用      | 提升表达能力、支持并行注意力建模        |
| 不改变    | 最终输出维度 `d_model`                 |
| 常见设置  | 一般是 2、4、8、12、16，需整除 d_model  |

---

要不要我帮你画一个图展示 **多头注意力的流程结构图**？或者直接跑一个例子展示不同 head 数对输出的影响？✨

你问的非常关键，这关系到 Transformer 和深度学习中张量（tensor）的标准形状定义。

下面我会详细解释：

---

## ✅ 首先，标准张量表示：[B, L, D]

| 符号 | 含义                             | 举例                            |
|------|----------------------------------|----------------------------------|
| `B`  | Batch size（批大小）              | 一次输入多少个样本，比如 32 条句子 |
| `L`  | Sequence length（序列长度）       | 每个样本有多少个 token（比如 10）|
| `D`  | Embedding dim（嵌入/特征维度）     | 每个 token 表示成几维向量（如 64）|

---

## 🧠 那为什么 `D` 是 "dimension"，而不是 "output layer" 呢？

这是因为：
> 在 Transformer 里，每个 token 会被表示成一个维度为 `D` 的向量，这个向量就是该 token 的上下文表示。

它 **不是模型的最终输出**，而是 Encoder/Attention 过程中每一步中间的“表示空间维度”。

---

### 举个形象的例子：

假设你输入了一句话：
```
"I love AI"
```

你先将它变成 token id：`[2, 8, 5]`

然后经过 `nn.Embedding(10000, 64)`，变成：

```python
[[ 0.12, 0.45, ..., -0.04],  ← token 2 ("I")
 [ 0.33, -0.17, ..., 0.91],  ← token 8 ("love")
 [ -0.03, 0.88, ..., -0.22]] ← token 5 ("AI")
```

这时候张量形状就是：

```
[B=1, L=3, D=64]
```

---

## 🧩 那 Output layer 在哪？

真正的 Output Layer 是像这样：

```python
nn.Linear(D, num_classes)   # 或 nn.Linear(D, 1)  用于分类或回归
```

这个才是你理解中的 “输出层”。

- Transformer 模块中的 D 是一种“隐藏维度”（hidden dimension）；
- 最终你会把它接入输出层（分类或回归）；
- 所以 D 是模型的**工作维度**，不是最终输出维度。

---

## 🔄 总结：D 是什么？

| 位置                      | D 的含义                  |
|---------------------------|---------------------------|
| `nn.Embedding(vocab, D)` | 每个 token 映射成 D 维向量 |
| `Transformer` 中间层       | 每个位置的特征维度         |
| `nn.Linear(D, ...)`      | D 是输入维度（linear输入）  |

---

## ✅ 一句话总结：

> D 是 Transformer 内部所有 token 表示的“向量空间维度”，不是模型的最终输出维度，但你最后会用它去 **输出分类 / 回归 / 预测结果**。

---

需要我帮你画一张张量流向图吗？或者举一个实际 batch 的例子给你跑一遍？让你看到 `[B, L, D]` 在每一步怎么变化 👀✨